# 🤖 Système RAG Complet avec Mistral AI

Chatbot conversationnel avec recherche sémantique et génération de réponses via **Mistral LLM**.

**Étapes :**

1. Chargement du système RAG
2. Configuration du LLM Mistral
3. Tests conversationnels
4. Évaluation qualitative


## 1️⃣ Configuration


In [1]:
import json
import numpy as np
from pathlib import Path
from typing import List, Dict

import faiss
from mistralai import Mistral

from src.config.constants import PROCESSED_DATA_DIR
from src.config.settings import settings

print("✅ Configuration OK")
print(f"🧠 Provider : {settings.embedding_provider}")
print(f"💬 LLM : {settings.llm_model}")

✅ Configuration OK
🧠 Provider : mistral
💬 LLM : mistral-small-latest


## 2️⃣ Système RAG avec Mistral


In [ ]:
# Importer la classe d'embeddings du notebook 03
from mistralai import Mistral
from sentence_transformers import SentenceTransformer


class EmbeddingGenerator:
    """Générateur d'embeddings unifié (copié du notebook 03)."""

    def __init__(self, provider: str = None, model_name: str = None, api_key: str = None):
        self.provider = provider or settings.embedding_provider
        self.model_name = model_name or settings.embedding_model

        if self.provider == "mistral":
            if not api_key and not settings.mistral_api_key:
                raise ValueError("Mistral API key required")
            self.client = Mistral(api_key=api_key or settings.mistral_api_key)
            self.dimension = 1024
        else:
            self.model = SentenceTransformer(self.model_name)
            self.dimension = self.model.get_sentence_embedding_dimension()

    def encode_single(self, text: str) -> np.ndarray:
        """Encode un seul texte."""
        if self.provider == "mistral":
            response = self.client.embeddings.create(model=self.model_name, inputs=[text])
            return np.array([response.data[0].embedding], dtype=np.float32)
        else:
            return self.model.encode([text], convert_to_numpy=True)


print("✅ Classe EmbeddingGenerator définie")

✅ Classe EmbeddingGenerator définie


In [4]:
class MistralEventsRAG:
    """Système RAG pour événements avec Mistral AI."""

    def __init__(self):
        print("🔄 Initialisation du système RAG...")

        # Chemins
        index_dir = PROCESSED_DATA_DIR / "faiss_index"

        # Charger configuration
        with open(index_dir / "config.json", "r") as f:
            self.config = json.load(f)

        # Charger index FAISS
        self.index = faiss.read_index(str(index_dir / "events.index"))
        print(f"  ✅ Index FAISS chargé ({self.index.ntotal} vecteurs)")

        # Charger documents
        documents_path = PROCESSED_DATA_DIR / "rag_documents.json"
        with open(documents_path, "r", encoding="utf-8") as f:
            self.documents = json.load(f)
        print(f"  ✅ {len(self.documents)} documents chargés")

        # Charger générateur d'embeddings
        self.embedding_generator = EmbeddingGenerator()
        print(f"  ✅ Générateur d'embeddings chargé")

        # Charger client Mistral pour le LLM
        self.mistral_client = Mistral(api_key=settings.mistral_api_key)
        print(f"  ✅ Client Mistral LLM chargé")

        print("\n✅ Système RAG prêt !")

    def search(self, query: str, top_k: int = 5) -> List[Dict]:
        """Recherche sémantique d'événements."""

        # Encoder la requête
        query_embedding = self.embedding_generator.encode_single(query)
        faiss.normalize_L2(query_embedding)

        # Recherche
        distances, indices = self.index.search(query_embedding, top_k)

        # Formater les résultats
        results = []
        for idx, dist in zip(indices[0], distances[0]):
            doc = self.documents[idx]
            results.append(
                {"document": doc, "similarity": float(1 - dist), "distance": float(dist)}
            )

        return results

    def generate_response(self, query: str, results: List[Dict]) -> str:
        """Génère une réponse conversationnelle avec Mistral LLM."""

        if not results:
            return "Je n'ai trouvé aucun événement correspondant à votre recherche. Pouvez-vous reformuler ou préciser votre demande ?"

        # Construire le contexte pour le LLM
        context = "Voici les événements pertinents :\n\n"
        for i, result in enumerate(results, 1):
            doc = result["document"]
            context += f"Événement {i}:\n{doc['content']}\n\n"

        # Prompt pour le LLM
        system_prompt = """Tu es un assistant spécialisé dans la recommandation d'événements culturels.
Ton rôle est d'aider les utilisateurs à trouver des événements qui correspondent à leurs intérêts.
Réponds de manière naturelle, conviviale et concise en français.
Base tes réponses uniquement sur les événements fournis dans le contexte."""

        user_prompt = f"""Question de l'utilisateur : {query}

{context}

Réponds à la question de l'utilisateur en te basant sur ces événements.
Présente 2-3 événements maximum de manière attrayante avec :
- Le titre
- La ville
- Une brève description
- Le lien si disponible

Sois concis et engageant."""

        # Appel à Mistral LLM
        response = self.mistral_client.chat.complete(
            model=settings.llm_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=settings.llm_temperature,
            max_tokens=settings.max_tokens,
        )

        return response.choices[0].message.content

    def chat(self, query: str, top_k: int = 5) -> str:
        """Interface conversationnelle complète."""
        results = self.search(query, top_k)
        return self.generate_response(query, results)


# Initialiser le système
print("🚀 Initialisation...\n")
rag = MistralEventsRAG()

🚀 Initialisation...

🔄 Initialisation du système RAG...
  ✅ Index FAISS chargé (497 vecteurs)
  ✅ 497 documents chargés
  ✅ Générateur d'embeddings chargé
  ✅ Client Mistral LLM chargé

✅ Système RAG prêt !


## 3️⃣ Tests Conversationnels


In [5]:
# Test 1 : Recherche par type
print("💬 User: Je cherche des concerts de jazz\n")
response = rag.chat("concerts de jazz")
print(f"🤖 Assistant:\n{response}")
print("\n" + "=" * 80 + "\n")

💬 User: Je cherche des concerts de jazz

🤖 Assistant:
Voici quelques concerts de jazz qui pourraient t'intéresser :

1. **Concert au musée Cantini avec Oliver Foster**
   - **Ville** : Marseille
   - **Description** : Un concert intimiste au musée Cantini, parfait pour les amateurs de jazz.
   - **Date** : 17/05/2025 de 17:00 à 20:00

2. **Black sound tricks & Friends**
   - **Ville** : Marseille
   - **Description** : Un événement festif pour la Fête de la Musique 2025, avec une programmation éclectique.
   - **Date** : 21/06/2025 de 10:00 à 22/06/2025 à 00:00

Profite bien de ces moments musicaux !




In [6]:
# Test 2 : Recherche par ville
print("💬 User: Quels événements à Marseille ce week-end ?\n")
response = rag.chat("événements à Marseille")
print(f"🤖 Assistant:\n{response}")
print("\n" + "=" * 80 + "\n")

💬 User: Quels événements à Marseille ce week-end ?

🤖 Assistant:
Voici quelques événements à ne pas manquer à Marseille :

1. **"Si Victor m'était conté"**
   - **Date** : Du 20/09/2025 à 18:30 au 21/09/2025 à 16:30
   - **Description** : Plongez dans un spectacle captivant mêlant contes, légendes chevaleresques, chants grégoriens et lyriques, danses et apparitions fantomatiques. Un voyage à travers quinze siècles d'histoire méditerranéenne.
   - **Adresse** : Place Saint-Victor, 13007 Marseille, France

2. **"À la rencontre de celles et ceux qui réinventent la Citadelle de Marseille"**
   - **Date** : Du 20/09/2025 à 10:00 au 21/09/2025 à 16:00
   - **Description** : Découvrez les projets innovants qui animent le fort Saint-Nicolas lors des Journées Européennes du Patrimoine 2025.
   - **Adresse** : Montée du Souvenir Français, 13007 Marseille

3. **"Veillée de paroles et musiques"**
   - **Date** : Du 21/07/2025 à 07:30 au 31/07/2025 à 15:30
   - **Description** : Rejoignez le collec

In [7]:
# Test 3 : Recherche pour enfants
print("💬 User: Des activités créatives pour mes enfants\n")
response = rag.chat("activités créatives pour enfants")
print(f"🤖 Assistant:\n{response}")
print("\n" + "=" * 80 + "\n")

💬 User: Des activités créatives pour mes enfants

🤖 Assistant:
Voici quelques activités créatives pour enfants à Marseille :

1. **Atelier pratiques artistiques**
   - **Ville** : Marseille
   - **Description** : Un projet de sensibilisation et d'expérimentation autour de la peinture et du volume avec l'argile, spécialement conçu pour les enfants.
   - **Adresse** : 5 Traverse Paul Converset 13014 Marseille

2. **Rouvrir le monde / Cadavre exquis animé / Clara Buffey**
   - **Ville** : Marseille
   - **Description** : Une expérience collaborative où les enfants créent ensemble un court métrage d'animation en combinant leurs dessins individuels.
   - **Adresse** : 7 Rue Pascal Posado, 13015 Marseille

3. **Ateliers d'écriture créative avec le Labo des histoires**
   - **Ville** : Marseille
   - **Description** : Des ateliers d'écriture créative pour les enfants, organisés en collaboration avec le Labo des histoires et l'auteur dramaturge Lionel Parrini.
   - **Adresse** : bd de la Corde

In [8]:
# Test 4 : Recherche culturelle
print("💬 User: Je m'intéresse à l'art contemporain\n")
response = rag.chat("expositions art contemporain")
print(f"🤖 Assistant:\n{response}")
print("\n" + "=" * 80 + "\n")

💬 User: Je m'intéresse à l'art contemporain

🤖 Assistant:
Voici quelques expositions d'art contemporain à ne pas manquer à Marseille :

1. **Exposition d'art contemporain**
   - **Ville** : Marseille
   - **Description** : Découvrez une galerie d'art contemporain avec un showroom permanent près du Vieux-Port. La galerie abrite également un atelier d'artistes.
   - **Adresse** : 25, cours d'Estienne d'Orves 13001 Marseille

2. **Friche la Belle de Mai : Les expositions d'art contemporain**
   - **Ville** : Marseille
   - **Description** : Plongez dans l'univers de Madison Bycroft avec l'exposition « Les mensonges du météorologue ».
   - **Adresse** : 41 rue Jobin 13003 Marseille

3. **Exposition "Hors Site (mais pas hors sol)"**
   - **Ville** : Marseille
   - **Description** : Une exposition fascinante sur la préfabrication en architecture.
   - **Adresse** : 12 boulevard Théodore Thurner 13006 Marseille

Profitez de ces événements culturels pour explorer l'art contemporain à Marseille

In [9]:
# Test 5 : Recherche gratuite
print("💬 User: Des événements gratuits ce week-end\n")
response = rag.chat("événements gratuits")
print(f"🤖 Assistant:\n{response}")
print("\n" + "=" * 80 + "\n")

💬 User: Des événements gratuits ce week-end

🤖 Assistant:
Voici quelques événements gratuits à ne pas manquer à Marseille :

1. **Les expositions en accès libre tout le weekend**
   - **Ville** : Marseille
   - **Description** : Profitez d'un accès gratuit aux expositions du J4 et du Fort Saint-Jean, comme « Lire le ciel » et « Amazighes ».
   - **Date** : Du 20/09/2025 à 08:00 au 21/09/2025 à 17:00

2. **Visites guidées gratuites à la découverte du CICRP**
   - **Ville** : Marseille
   - **Description** : Découvrez les bâtiments et les ateliers de restauration du Centre Interdisciplinaire de Conservation et de Restauration du Patrimoine.
   - **Date** : Du 20/09/2025 à 07:30 au 20/09/2025 à 14:00

3. **Veillée de paroles et musiques**
   - **Ville** : Marseille
   - **Description** : Une veillée partagée avec le collectif Transbordeur pour un voyage à la rencontre des habitants d'un quartier.
   - **Date** : Du 21/07/2025 à 07:30 au 31/07/2025 à 15:30

Profitez de ces événements pour 

## 4️⃣ Interface Interactive


In [10]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# Créer l'interface
query_input = widgets.Text(
    placeholder="Posez votre question...",
    description="Question:",
    layout=widgets.Layout(width="80%"),
)

search_button = widgets.Button(description="🔍 Rechercher", button_style="primary")

output_area = widgets.Output()


def on_search_click(b):
    with output_area:
        clear_output()
        query = query_input.value

        if query:
            print(f"💬 Recherche : {query}\n")
            print("⏳ Génération de la réponse...\n")

            response = rag.chat(query, top_k=5)
            print("🤖 Assistant :")
            display(Markdown(response))
        else:
            print("⚠️ Veuillez saisir une question")


search_button.on_click(on_search_click)

# Afficher l'interface
display(
    widgets.VBox(
        [
            widgets.HTML("<h2>🤖 Assistant Événements Mistral</h2>"),
            query_input,
            search_button,
            output_area,
        ]
    )
)

## 5️⃣ Métriques de Performance


In [12]:
import time

# Benchmark end-to-end
queries = [
    "concert jazz",
    "expo art",
    "théâtre enfants",
]

times = []
for query in queries:
    print(f"⏱️ Test : {query}")
    start = time.time()
    _ = rag.chat(query)
    elapsed = time.time() - start
    times.append(elapsed)
    print(f"   → {elapsed:.2f}s\n")

print("\n⚡ Performance système complet :")
print(f"  Temps moyen : {np.mean(times):.2f} s")
print(f"  Temps min : {min(times):.2f} s")
print(f"  Temps max : {max(times):.2f} s")

⏱️ Test : concert jazz
   → 2.34s

⏱️ Test : expo art
   → 2.54s

⏱️ Test : théâtre enfants
   → 2.02s


⚡ Performance système complet :
  Temps moyen : 2.30 s
  Temps min : 2.02 s
  Temps max : 2.54 s


## ✅ Résumé

### Points forts du système

✅ **Embeddings Mistral** : Vecteurs de haute qualité (1024 dimensions)  
✅ **LLM Mistral** : Réponses naturelles et contextuelles  
✅ **Recherche sémantique** : Pertinence élevée  
✅ **API unique** : Un seul provider pour tout le pipeline

### Avantages Mistral AI

- 🇫🇷 Optimisé pour le français
- 🚀 Réponses rapides et cohérentes
- 🔒 Hébergement européen (conformité RGPD)
- 💰 Tarification compétitive
